# Try Azure ML Studio with a Service Principal
This interactive Jupyter Notebook will allow you to setup infrastructure on Azure ML and authenticate using a Service Principal. 

Make sure you have enabled the following GitHub repository secrets:

```
SUBSCRIPTION_ID
AZURE_SERVICE_PRINCIPAL_TENANT
AZURE_SERVICE_PRINCIPAL_APPID
AZURE_SERVICE_PRINCIPAL_PASSWORD
```

Optionally, you can add the following items to use a specific resource group, region, or existing workspace:

```
WORKSPACE_REGION
RESOURCE_GROUP
WORKSPACE_NAME
```

These are retrieved and set from the next cell

In [6]:
import os

# required
subscription_id = os.environ["SUBSCRIPTION_ID"]
tenant_id = os.environ["TENANT_ID"]
service_principal_id = os.environ["AZURE_SERVICE_PRINCIPAL_APPID"]
service_principal_password = os.environ["AZURE_SERVICE_PRINCIPAL_PASSWORD"]

# optional
resource_group = os.getenv("RESOURCE_GROUP", default="demo-try-azureml")
workspace_name = os.getenv("WORKSPACE_NAME", default="demo-try-azureml")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")


## Define the Service Principal 
Use the environment variables to instantiate the Service Principal account so that it can be used to authenticate to Azure later

In [7]:
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication

service_principal = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password=service_principal_password
)


## Create the workspace 
Use the previously created configuration file to create the Azure ML workspace

In [8]:
# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      auth=service_principal,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

Deploying StorageAccount with name demotryastoragedc4163c49.
Deploying KeyVault with name demotryakeyvault8f7f8fda.
Deploying AppInsights with name demotryainsights67ca0475.
Deployed AppInsights with name demotryainsights67ca0475. Took 1.29 seconds.
Deployed KeyVault with name demotryakeyvault8f7f8fda. Took 16.59 seconds.
Deploying Workspace with name demo-try-azureml.
Deployed StorageAccount with name demotryastoragedc4163c49. Took 21.88 seconds.
Deployed Workspace with name demo-try-azureml. Took 18.57 seconds.


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute

# define aml compute target(s) to create
amlcomputes = {
    "cpu-cluster": {
        "vm_size": "STANDARD_DS3_V2",
        "min_nodes": 0,
        "max_nodes": 3,
        "idle_seconds_before_scaledown": 240,
    }
}

# create aml compute targets
for ct_name in amlcomputes:
    if ct_name not in ws.compute_targets:
        compute_config = AmlCompute.provisioning_configuration(**amlcomputes[ct_name])
        ct = ComputeTarget.create(ws, ct_name, compute_config)
        ct.wait_for_completion(show_output=True)

InProgress........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Perform exploratory work
You can now interact with the Azure ML Workspace, like train models, register datasets and models, or create an AutoML run

In [ ]:
# exploratory work

## Destroy workspace and resources
The next cell will destroy all assets from the Azure ML workspace

In [10]:
ws.delete(delete_dependent_resources=True, no_wait=False)